## 用HC方法对数据nci60关于renal和nsclc进行分类

In [11]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import scipy

In [12]:
data=pd.read_csv('./data/NCI60.csv',index_col=0)
data.head()

,data.1,data.2,data.3,data.4,data.5,data.6,data.7,data.8,data.9,data.10,...,data.6822,data.6823,data.6824,data.6825,data.6826,data.6827,data.6828,data.6829,data.6830,labs
V1,0.300000,1.180000,0.550000,1.140000,-0.265000,-7.000000e-02,0.350000,-0.315000,-0.450000,-0.654981,...,0.000000,0.030000,-0.175000,0.629981,-0.030000,0.000000,0.280000,-0.340000,-1.930000,CNS
V2,0.679961,1.289961,0.169961,0.379961,0.464961,5.799610e-01,0.699961,0.724961,-0.040039,-0.285020,...,-0.300039,-0.250039,-0.535039,0.109941,-0.860039,-1.250049,-0.770039,-0.390039,-2.000039,CNS
V3,0.940000,-0.040000,-0.170000,-0.040000,-0.605000,0.000000e+00,0.090000,0.645000,0.430000,0.475019,...,0.120000,-0.740000,-0.595000,-0.270020,-0.150000,0.000000,-0.120000,-0.410000,0.000000,CNS
V4,0.280000,-0.310000,0.680000,-0.810000,0.625000,-1.387779e-17,0.170000,0.245000,0.020000,0.095019,...,-0.110000,-0.160000,0.095000,-0.350020,-0.300000,-1.150010,1.090000,-0.260000,-1.100000,RENAL
V5,0.485000,-0.465000,0.395000,0.905000,0.200000,-5.000000e-03,0.085000,0.110000,0.235000,1.490019,...,-0.775000,-0.515000,-0.320000,0.634980,0.605000,0.000000,0.745000,0.425000,0.145000,BREAST


In [13]:
y=data.loc[:,'labs']
y1=y[y=='RENAL']
y2=y[y=='NSCLC']
sample=data.columns
xnames=data.index
df1=data[data['labs']=='RENAL']
df2=data[data['labs']=='NSCLC']
df1.loc[:,'labs']=np.ones(len(df1))
df2.loc[:,'labs']=-1*np.ones(len(df2))
df3 = pd.concat([df1,df2],axis=0,ignore_index=False)
data1=df3.iloc[:,:-1]
#npdata=preprocessing.scale(data.iloc[:,:-1])
data_scale=data1.apply(lambda x:(x-np.mean(x)/np.std(x)))
newdata=pd.concat([data_scale,df3.iloc[:,-1]],axis=1,ignore_index=False)
df11=newdata[newdata['labs']==1]
df22=newdata[newdata['labs']==-1]
df33 = pd.concat([df11,df22],axis=0,ignore_index=False)
#df11=df11.apply(lambda x:(x-np.mean(x)/np.std(x)))
#df22=df2.apply(lambda x:(x-np.mean(x)/np.std(x)))
#df1_scaled = preprocessing.scale(df11)
#df2_scaled = preprocessing.scale(df22)

C:\jupy\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [14]:
#Z统计量
n=len(data.index)
Z=[]
for i in range(len(data.columns)-1):
    Z.append((df11.iloc[:,i].sum()-df22.iloc[:,i].sum())*n**(-0.5))


In [15]:
#P值
p_values=[]
for i in range(len(Z)):
    #p_values.append(scipy.stats.norm.sf(abs(Z[i]))*2)
    p_values.append((1-scipy.stats.norm.cdf(Z[i]))/2)
p_values.sort()

In [16]:
#HC统计量
HC=[]
N=len(Z)
for i in range(len(Z)-1):
    HC.append(N**(0.5)*((i+1)/N-p_values[i])/((i+1)/N-((i+1)/N)**2)**(-0.5))
alpha=0.1
HC1=HC[:int(N*alpha)]
MAX=HC1.index(max(HC1))
MAX

13

### 用论文中定义的统计量进行筛选

In [17]:
#选取变量 
Z1=list(enumerate(Z))
Z1.sort(key=lambda x:x[1],reverse=True)
Z1[:(MAX+1)]
#选取的变量为：
index=[]
for i in range(14):
    
    index.append('data.'+str(Z1[:(MAX+1)][i][0]))
print(index)

['data.5979', 'data.5941', 'data.6086', 'data.6242', 'data.127', 'data.6085', 'data.4699', 'data.6084', 'data.5978', 'data.6083', 'data.4698', 'data.5898', 'data.5804', 'data.5980']


In [19]:
t=Z1[(MAX)][1]
def sgn(z):
    if z>0:
        return 1
    elif z==0:
        return 0
    else:
        return 1
    
def soft(z):
    if abs(z)>t:
        return sgn(z)*z
    else:
        return 0
softdb=[]
Z1=[]
for i in Z:
    Z1.append(soft(i))
#decision based
npz=np.array(Z1)
for i in range(len(df33)):
    L=np.dot(npz,df33.iloc[i,:-1])
    if L>0:
        softdb.append(1)
    else:
        softdb.append(-1)

In [20]:
#分类结果
print('分类结果',softdb)
#错分率：
sum(softdb!=df33.iloc[:,-1])/len(df33)

分类结果 [1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1]


0.05555555555555555